In [2]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from audio_proc_utils import *

/home/rbaijal/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [ ]:
class LinearClassifier:
    def __init__(self):
        self.W =None # define model parameters here
    
    def softmax(self, z):
        z1 = np.exp(z)
        return z1 / np.sum(z1, axis=0)
    
    def y_hat(self, X):
        z = self.W @ X
        return self.softmax(z)
    
    def cross_entropy_loss(self, y_pred, y ):
        # y is of shape (3 x num_frames)
        loss = -np.sum((np.log(y_pred) * y))
        loss = loss / y.shape[1]
    
    def predict(self, X):
        yhat = self.y_hat(X)
        predicted = yhat.argmax(axis=0)
        return predicted
    
    def train(self, X, Y):
        loss_history = []
        
        for i in range(epochs):
            y_pred = self.y_hat(X)

            grad = (y_pred - Y) @ X.T
            N = X.shape[1]
            grad = grad/N
            
            self.W = self.W - alpha * grad
            
            l = self.cross_entropy_loss(y_pred, Y)
            loss_history.append(l)
        return loss_history
    

In [41]:
a = np.array([[0,1,0], [0.3, 0.3, 0.4]]).T
z1 = np.exp(a)
print(z1)
s = z1 / np.sum(z1, axis=0)
t = np.array([[0,1,0], [1,0,0]]).T
print(s)
b = s*t
print(b)
print(np.sum(b, axis=1))

[[1.         1.34985881]
 [2.71828183 1.34985881]
 [1.         1.4918247 ]]
[[0.21194156 0.32204346]
 [0.57611688 0.32204346]
 [0.21194156 0.35591307]]
[[0.         0.32204346]
 [0.57611688 0.        ]
 [0.         0.        ]]
[0.32204346 0.57611688 0.        ]


In [3]:
Fs= 16000
n_mfcc = 20

# Read audio
x_music = readDir('music_speech/music_wav', Fs)    #change it as per your directory
x_speech = readDir('music_speech/speech_wav', Fs)  #change it as per your directory
x_silence = readDir('music_speech/silence_wav', Fs)
X = np.concatenate((x_music, x_speech, x_silence))

# Create labels
y_music = np.array([[1,0,0]]*len(x_music))
y_speech = np.array([[0,1,0]]*len(x_speech))
y_silence = np.array([[0,0,1]]*len(x_silence))
Y = np.concatenate((y_music, y_speech, y_silence))

X_train, y_train, X_test, y_test = splitData(X, Y)

In [4]:
x_train = audio2mfcc(X_train, n_mfcc, Fs)    # x_train: (Nclips, N_mfcc, N_frames)

In [ ]:
x1 = x_train[0]
x2 = x_train[1]
x1 = x1.T
x_train.shape

(484, 20, 313)

In [6]:
a1 = np.array([1,2,3])
a2 = np.array([2,3,4])
a3 = np.array([5,6,7])
a = []
a.append(a1)
a.append(a2)
a.append(a3)
ans = np.vstack(a)
ans.T

array([[1, 2, 5],
       [2, 3, 6],
       [3, 4, 7]])

In [7]:
np.hstack((y_train[0].reshape(-1,1), y_train[1].reshape(-1,1)))

array([[0, 1],
       [0, 0],
       [1, 0]])

In [8]:
X_train.shape

(484, 160000)

In [14]:
def unwrap_data(X, Y):
    x = []
    y = []
    # X is of shape (num_clips x num_features x num_frames)
    for i in range(X.shape[0]):
        x1 = X[0].T
        for j in range(x1.shape[0]):
            x.append(x1[j])
            y.append(Y[i])
    
    x = np.vstack(x)
    y = np.vstack(y)
    return x,y

In [15]:
x, y = unwrap_data(x_train, y_train)